# Project: Fraud Detection 

## 1. Overview

### PaySim simulates mobile money transactions based on a sample of real transacions extracted from one month of financial logs from a mobile money service implemented in an African country. The original logs were provided by a multinational company, who is the provider of the mobile financial service which is currently running in more than 14 countries all around the world. The objective of the project is to predict if a transaction is fraudulent or not.

## 2. Preprocess the data

#### We´ll use PySpark to preprocess the data.

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from  pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
import functools
import seaborn as sb
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as mpt
from pyspark.sql.types import IntegerType

#from sklearn.datasets import make_classification

In [ ]:
# global variables

global df_bank, results 

In [ ]:
# creation of the SparkSession

spark = SparkSession.builder.appName("FraudDetection").getOrCreate()
spark

In [ ]:
# spark dataframe 

df = spark.read.csv('fraudDetection.csv', header=True)

#### Let´s take a look to the data with the first 10 rows.

In [ ]:
df.show(10)

In [ ]:
df.printSchema()

#### There are 11 columns, some of them are numerical and others are categorical. Let´s count the number of registers.

In [ ]:
print(f"The total number of registers is:",df.count())

#### We have more than six miliions of transactions in the dataset.

### 2.1 Feature Engineering

#### Firstly, we´ll create a function to create a new variable.

In [ ]:
### 2.1.1.- creation of a new variable: type2

df_type2 = df.withColumn("type2",f.concat(f.substring("nameOrig",1,1),f.substring("nameDest",1,1)))

In [ ]:
df_type2.show(5)

#### We´ve created a new column named "type2" which is composed by the first character of the column "nameOrig" and the first character of the column "nameDest"

In [ ]:
### 2.1.2.1.- One Hot Encoding: column "type"

df_type2.show(3)

#### We´ll use some libraries of Spark for Machine Learning (SparkML).

In [ ]:
### StringIndexer Initialization
### column: type

indexer_type = StringIndexer(inputCol="type",outputCol="types_indexed")
indexerModel_type = indexer_type.fit(df_type2)


In [ ]:
### Transform the DataFrame using the fitted StringIndexer model

indexed_df_type2 = indexerModel_type.transform(df_type2)
indexed_df_type2.show(10)

#### Here, we´ve set each of the elements of the "type" column into indexes.

In [ ]:
### apply One-Hot-Encoding to the indexed column, that is, 
### "types_indexed"

encoder_type = OneHotEncoder(dropLast=False, inputCol="types_indexed", outputCol="types_onehot")
encoder_type_df = encoder_type.fit(indexed_df_type2).transform(indexed_df_type2)
encoder_type_df.show(truncate=False)


In [ ]:
encoder_type_df.printSchema()

In [ ]:
encoder_type_df_split = encoder_type_df.select('*',vector_to_array('types_onehot').alias('types_onehot_split'))
encoder_type_df_split.show(5)

In [ ]:
### now, we´ll split the "types_onehot_split" into five columns, one per category

num_categories = len(encoder_type_df_split.first()['types_onehot_split'])
cols_expanded = [(f.col('types_onehot_split')[i].alias(f"{indexerModel_type.labels[i]}")) for i in range(num_categories)]
type_df = encoder_type_df_split.select('*',*cols_expanded)


In [ ]:
type_df.show(100)

#### We´ve applied One-Hot-Encoding to the column "type" resulting in five new columns:
+ CASH_OUT
+ CASH_IN
+ PAYMENT
+ TRANSFER 
+ DEBIT

#### Now, we´ll apply this procedure to the column "type2".

In [ ]:
### 2.1.2.2.- One Hot Encoding: column "type2"

type_df.show(5)

In [ ]:
### StringIndexer Initialization
### column: type2

indexer_type = StringIndexer(inputCol="type2",outputCol="types_indexed2")
indexerModel_type = indexer_type.fit(type_df)

In [ ]:
### Transform the DataFrame using the fitted StringIndexer model

indexed_df_type = indexerModel_type.transform(type_df)
indexed_df_type.show(10)

In [ ]:
### apply One-Hot-Encoding to the indexed column, that is, 
### "types_indexed2"

encoder_type2 = OneHotEncoder(dropLast=False, inputCol="types_indexed2", outputCol="types_onehot2")
encoder_type2_df = encoder_type2.fit(indexed_df_type).transform(indexed_df_type)
encoder_type2_df.show(truncate=False)

In [ ]:
encoder_type2_df.printSchema()

In [ ]:
encoder_type2_df_split = encoder_type2_df.select('*',vector_to_array('types_onehot2').alias('types_onehot_split2'))
encoder_type2_df_split.show(5)

In [ ]:
### now, we´ll split the "types_onehot_split2" into two columns, one per category

num_categories = len(encoder_type2_df_split.first()['types_onehot_split2'])
cols_expanded = [(f.col('types_onehot_split2')[i].alias(f"{indexerModel_type.labels[i]}")) for i in range(num_categories)]
encoder_type2_df_split = encoder_type2_df_split.select('*',*cols_expanded)

In [ ]:
encoder_type2_df_split.show(5)

#### We´ve split the "type2" column into two columns based on One-Hot-Encoding. Now, we´ll eliminate some unnecessaruy columns. Let´s check out all the columns.

In [ ]:
encoder_type2_df_split.printSchema()

In [ ]:
### 2.1.3.- Eliminate unneccesary columns: "nameOrig","nameDest","isFlaggedFraud","newbalanceDest",
### "oldbalanceDest","oldbalanceOrg","newbalanceOrig","types_indexed","types_onehot","types_onehot_split",
### "types_indexed2","types_onehot2", "types_onehot_split2"


#### Now, we´ll eliminate the unnecessary columns:
+ nameOrig
+ nameDest
+ isFlaggedFraud
+ newbalanceDest
+ oldbalanceDest
+ oldbalanceOrg
+ newbalanceOrig 
+ types_indexed
+ types_onehot
+ types_onehot_split
+ types_indexed2
+ types_onehot2
+ types_onehot_split2
+ type
+ type2

In [ ]:
df_bank = encoder_type2_df_split.drop("nameOrig","nameDest","isFlaggedFraud","newbalanceDest","oldbalanceDest",
                       "oldbalanceOrg","newbalanceOrig","type","types_indexed","types_onehot",
                       "types_onehot_split","type2","types_indexed2","types_onehot2","types_onehot_split2" )
df_bank.show(5)

In [ ]:
df_bank.count()

#### We can see that there are the same quantity of registers.

### 2.2 Data Cleaning

In [ ]:
### 2.2.1.- Eliminate duplicated

num_all_rows = df_bank.count()
num_all_rows

In [ ]:
num_duplicated_rows = df_bank.distinct().count() 

In [ ]:
print(f"The total number of duplicated rows is:",num_all_rows - num_duplicated_rows)

#### We can see that there are 7597 duplicated rows.

In [ ]:
df_bank = df_bank.dropDuplicates()

In [ ]:
df_bank.count()

#### We can see the duplicated registers have been removed because there fewer registers than before.

In [ ]:
### 2.2.2.- Eliminate null values

df_bank.dropna()

In [ ]:
df_bank.count()

#### We can see that there were no null values in the dataset because the number of registers is the same. Let´s take a look at the "clean" dataset.

In [ ]:
df_bank.show(10)

## 3. Exploratory Data Analysis (EDA)

### 3.1 Visualization

#### The visualization will be done using a functions which leverages the method histogram() of pyspark. 

In [ ]:
# definition of the "histogram" function

def histogram(df, col, bins=10, xname=None, yname=None):
    
    '''
    This function makes a histogram from spark dataframe named 
    df for column name col. 
    '''
    
    # Calculating histogram in Spark 
    vals = df.select(col).rdd.flatMap(lambda x: x).histogram(bins)
    
    # Preprocessing histogram points and locations 
    width = vals[0][1] - vals[0][0]
    loc = [vals[0][0] + (i+1) * width for i in range(len(vals[1]))]
    
    # Making a bar plot 
    mpt.bar(loc, vals[1], width=width)
    mpt.xlabel(col)
    mpt.ylabel(yname)
    mpt.show()

In [ ]:
df_bank.printSchema()

#### There are some features that need to be converted to integers such as "step","amount" and "isFraud".

In [ ]:
# convert string columns into integer columns

df_bank = df_bank.withColumn("step",df_bank["step"].cast(IntegerType()))

In [ ]:
df_bank = df_bank.withColumn("amount",df_bank["amount"].cast(IntegerType()))

In [ ]:
df_bank = df_bank.withColumn("isFraud",df_bank["isFraud"].cast(IntegerType()))

In [ ]:
df_bank.printSchema()

#### We´ve seen that all the features are "integer" types now. Therefore, we´re able to perform various visualizations with the histogram method. That´s what we´ll do next.

In [ ]:
# histogram: "step"

histogram(df_bank, 'step', bins=15, yname='frequency')

In [ ]:
# histogram: "amount"

histogram(df_bank, 'amount', bins=15, yname='frequency')

In [ ]:
# histogram: "Debit"

histogram(df_bank, 'Debit', bins=15, yname='frequency')


In [ ]:
# histogram: "Payment"

histogram(df_bank, 'Payment', bins=15, yname='frequency')


In [ ]:
# histogram: "CASH_OUT"

histogram(df_bank, 'CASH_OUT', bins=15, yname='frequency')


In [ ]:
# histogram: "CASH_IN"

histogram(df_bank, 'CASH_IN', bins=15, yname='frequency')


In [ ]:
# histogram: "TRANSFER"

histogram(df_bank, 'TRANSFER', bins=15, yname='frequency')


In [ ]:
# histogram: "CC"

histogram(df_bank, 'CC', bins=15, yname='frequency')


In [ ]:
# histogram: "CM"

histogram(df_bank, 'CM', bins=15, yname='frequency')

In [ ]:
# histogram: "isFraud"

histogram(df_bank, 'isFraud', bins=15, yname='frequency')

#### Remember that our label is "isFraud", therefore, we can see that this class is unbalanced as we can see from the previous graphic. We need to perform an **Oversampling**.

In [ ]:
### oversampling with "pysaprk"

minor_df = df_bank.filter(f.col("isFraud")==1)
major_df = df_bank.filter(f.col("isFraud")==0)

In [ ]:
ratio = int(major_df.count()/minor_df.count())

In [ ]:
print("The ratio is:",ratio)

In [ ]:
a = range(ratio)

In [ ]:
# let´s duplicate the minoriry rows

oversampled_df = minor_df.withColumn("dummy",f.explode(f.array([f.lit(x) for x in a]))).drop("dummy")

In [ ]:
oversampled_df.show(5)

In [ ]:
oversampled_df.printSchema()

In [ ]:
minor_df = oversampled_df.filter(f.col("isFraud")==1)

In [ ]:
oversampled_df.count()

In [ ]:
major_df.count()

In [ ]:
combined_df = major_df.unionAll(oversampled_df)
combined_df.show(5)

In [ ]:
combined_df.count()

In [ ]:
combined_df.dropDuplicates()

In [ ]:
combined_df.count()

In [ ]:
minor_df = combined_df.filter(f.col("isFraud")==1)
major_df = combined_df.filter(f.col("isFraud")==0)

In [ ]:
minor_df.show(50)

In [ ]:
major_df.show(50)

In [ ]:
minor_df.count()

In [ ]:
major_df.count()

In [ ]:
# now, we need to aggregate indexes to the "minor_df" dataframe

minor_df = minor_df.withColumn("index",monotonically_increasing_id())
minor_df.show(5)

In [ ]:
num_df_bank = df_bank.count()

In [ ]:
num_df_bank

In [ ]:
num_minor_df = round(num_df_bank/2)

In [ ]:
num_minor_df

In [ ]:
num_major_df = num_df_bank - num_minor_df

In [ ]:
num_major_df

In [ ]:
num_major_df + num_minor_df

In [ ]:
# now, we need to establish which rows must be deleted
# dataframe: minor_df

rows_to_remove = []
for i in range(num_minor_df):
    rows_to_remove.append(i)

In [ ]:
len(rows_to_remove)

In [289]:
minor_df = minor_df.filter(~minor_df.index.isin(rows_to_remove))

In [ ]:
minor_df.count()

In [ ]:
minor_df.show(50)

In [ ]:
# now, we need to establish which rows must be deleted
# dataframe: major_df

In [ ]:
type(rows_to_remove)

In [ ]:
rows_to_remove[0]

In [ ]:
combined_df = combined_df.filter(~combined_df.index.isin(rows_to_remove))

In [ ]:
combined_df.show(10)

In [ ]:
combined_df.count()

In [ ]:
minor_df = combined_df.filter(f.col("isFraud")==1)
major_df = combined_df.filter(f.col("isFraud")==0)

In [ ]:
minor_df.show(20)

In [ ]:
major_df.show(20)

In [ ]:
minor_df.count()

In [ ]:
major_df.count()

In [ ]:
rows_to_remove = [0:10]

In [ ]:
combined_df = combined_df.filter(~combined_df.index.isin(rows_to_remove))

In [ ]:
combined_df.show(20)

#### Now, we´ll transform it to a Pandas dataframe.

In [ ]:
df_bank_partitioned_pd = df_bank_partitioned.toPandas()

In [ ]:
type(df_bank_partitioned)

In [ ]:
counter = Counter(df_bank_partitioned_pd['isFraud'])

In [ ]:
counter

In [ ]:
df_bank_partitioned_pd.describe()

In [ ]:
df_bank_partitioned_pd.head(5)

In [ ]:
df_bank_partitioned_pd.info()

#### From the previous view, we can see that the columns have only "object" type. We need to cast it into integers using the method "astype()".

In [ ]:
df_bank_partitioned_pd['isFraud'] = df_bank_partitioned_pd['isFraud'].astype(int)

In [ ]:
df_bank_partitioned_pd['isFraud']

#### We can see that the type has been changed from "object" to "int64" (integer).

In [ ]:
df_bank_partitioned_pd.plot.hist()

In [ ]:
#
len(df_bank_partitioned_pd)


#### From the previous plot we can see that the classes are unbalanced. Remember that "0" means that there is no fraud and "1" is the opposite case. We need to apply a technique to revert this. We´ll use an algorithm named "SMOTE". 

In [ ]:
# create an object "SMOTE"

smt = SMOTE()

#### Let´s split the dataset into two pieces: "X" and "Y". 

In [ ]:
X_train, X_test,y_train,y_test = train_test_split(df_bank_partitioned_pd,df_bank_partitioned_pd,test_size=0.4)

In [ ]:
X_train, X_test

In [ ]:
len(X_train),len(y_train)

In [ ]:
len(X_test),len(y_test)

#### Suming both results in 6.35

In [ ]:
X_resampled, Y_resambled = smt.fit_resample(X_train,y_train)

In [ ]:
smt.fit

In [ ]:
type(X_resampled)

In [ ]:
len(X_resampled)

In [ ]:
Counter(df_bank_partitioned_pd['isFraud'])

In [ ]:
Counter(X_resampled['isFraud'])

In [ ]:
Counter(Y_resambled['isFraud'])

In [ ]:
X_resampled

In [ ]:
len(X_resampled)

In [ ]:
Y_resambled

In [ ]:
X_resampled.hist()

In [ ]:
X_resampled.count()

In [ ]:
Counter(X_resampled['isFraud'])

In [ ]:
len(X_resampled)

In [ ]:
len(Y_resambled)

In [ ]:
df_bank_partitioned_pd['isFraud']

In [ ]:
values0,value1 = df_bank_partitioned_pd.isFraud.value_counts()

In [ ]:
values0,value1

In [ ]:
df_class_0 = df_bank_partitioned_pd[df_bank_partitioned_pd['isFraud']==0]
df_class_1 = df_bank_partitioned_pd[df_bank_partitioned_pd['isFraud']==1]


In [ ]:
df_class_0, df_class_1 

In [ ]:
### Random over-sampling

df_class_1_overSampling = df_class_1.sample(values0, replace=True)

In [ ]:
df_class_1_overSampling

In [ ]:
df_test_overSampling = pd.concat([df_class_0,df_class_1_overSampling],axis=0)

In [ ]:
type(df_test_overSampling)

In [ ]:
df_test_overSampling.isFraud.value_counts()

In [ ]:
df_test_overSampling.hist()

In [ ]:
### RandonOverSampler (inmlearn)

ros = RandomOverSampler()


In [ ]:
X_ros , y_ros = ros.fit_resample(X_train,X_test)